In [ ]:
## download by REJ
import requests
import os
import json
import pprint as pp
import time
from random import randint
from time import sleep
import urllib.request

ZONE4_path = r'./ZONE4/'
for i in range(102, 110):
    if not os.path.exists(ZONE4_path):
        os.mkdir(ZONE4_path)
    if not os.path.exists("%s/%s" % (ZONE4_path, i)):
        os.mkdir("%s/%s" % (ZONE4_path, i))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'}

url_start = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=5f1da434-ad44-4955-97d3-1ac4433603f1(203.204.81.115)'
# url_start = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=f61d5b9e-0eb7-4160-86c9-ab76e064cb8c(111.249.61.96)'
url_each = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp'

dkind = ["REJ"]  # "REJ" over 130K
paths = [ZONE4_path]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
years = [102, 103, 104, 105, 106, 107, 108, 109]
m31 = [1, 3, 5, 7, 8, 10, 12]
m30 = [4, 6, 9, 11]
m29 = [2]
days_sta = [1,11,21,26]
data_REJ = {}
data_REJ_back = {}
# ZONE4_targets = [399,349,384,359,307,386,384,273]
# ZONE4_targets_back = [496,410,374,381,342,429,358,192]

keys = ["title", "kind", "markname", "content"]

for num in range(1):
    # select type of judgement.
    print("now is : kind", dkind[num])

    # year period select
    for year in range(8):
        try:
            for month in range(12):
                print("now",month,"mon", months[month])
                if months[month] in m31:
                    lastday = 31
                    days_end = [10, 20, 25, 31]
                    for dayend,daysta in zip(range(4),range(4)):
                        data_REJ = {'state': 'queryResultJSONObject',
                                         'searchConditions': '{"datePubStart":"%s/%s/%s","datePubEnd":"%s/%s/%s","dptKind":"%s","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'
                                        %(years[year], months[month],days_sta[daysta],years[year], months[month],days_end[dayend],dkind[num])
                                         }
                        res_REJ = requests.post(url_start, headers=headers, data=data_REJ)
                        list_REJ = json.loads(res_REJ.text)
                        item_REJ = 0
                        
                        for i in list_REJ['receiveData']['resultArr']:
                            d1 = {}
                            try:
                                kind_REJ = i['DPT_KIND']
                                mark_REJ = i['MARK_NAME']
                                no_REJ = i['PUB_NO']
                                title_REJ = no_REJ[-12:-1]
                                file_REJ = i['FILENAME']
                                data_REJ  = {'state': 'getContentByFileName',
                                            'filename': file_REJ
                                             }
    
                                # crawler content
                                res_sec_REJ = requests.post(url_each, headers=headers, data=data_REJ)
                                raw_content_REJ = json.loads(res_sec_REJ.text)
                                content_REJ = raw_content_REJ['content']['content']
    
                                #key:value
                                values = [title_REJ,kind_REJ,mark_REJ,content_REJ]
                                for n in range(len(keys)):
                                    d1[keys[n]]=values[n]
    
                                #save jsonfile
                                try:
                                    with open('%s/%s.json' %(paths[num],years[year]), 'a', encoding='utf-8') as f:
                                        json.dump(d1, f, ensure_ascii=False)
                                        f.write('\n')
                                except FileNotFoundError as e:
                                    no_REJ = no_REJ.replace('/', '-')
                                    with open('%s/%s.json' %(paths[num],years[year]), 'a', encoding='utf-8') as f:
                                        json.dump(d1, f, ensure_ascii=False)
                                        f.write('\n')
                                except OSError:
                                    pass
                                print("save json complete!===========================================================")
    
                                #print result
                                item_REJ += 1
                                if dkind[num] == "REJ":
                                    print(dkind[num],years[year],'REJ: ',item_REJ, title_REJ)
                                    
                                #sleep
                                sleep(randint(5,7))
    
                            except TypeError as e:
                                print('==========')
                                print(title_REJ,': no content found')
                                print(e.args)
                                print('==========')
                                continue
                            except json.JSONDecodeError as e:
                                print('==========')
                                print(title_REJ, ':no page found')
                                print(e.args)
                                print('==========')
                                continue
    
                        #sleep range
                        sleep(randint(10,15))

                #date_30
                elif months[month] in m30:
                    lastday = 30
                    days_end = [10, 20, 25, 30]
                    for dayend,daysta in zip(range(4),range(4)):
                        data_REJ = {'state': 'queryResultJSONObject',
                                         'searchConditions': '{"datePubStart":"%s/%s/%s","datePubEnd":"%s/%s/%s","dptKind":"%s","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'
                                        % (years[year], months[month], days_sta[daysta], years[year],months[month], days_end[dayend], dkind[num])
                                         }
                        res_REJ = requests.post(url_start, headers=headers, data=data_REJ)
                        list_REJ = json.loads(res_REJ.text)
                        #                     pp.pprint(list_REJ)
                        print('totalnum', years[year], "month:",months[month],"section:",daysta,list_REJ['receiveData']['totalNum'])
                        item_REJ = 0
                        
                        for i in list_REJ['receiveData']['resultArr']:
                            d1 = {}
                            try:
                                kind_REJ = i['DPT_KIND']
                                mark_REJ = i['MARK_NAME']
                                no_REJ = i['PUB_NO']
                                title_REJ = no_REJ[-12:-1]
                                file_REJ = i['FILENAME']
                                data_REJ = {'state': 'getContentByFileName',
                                            'filename': file_REJ
                                            }

                                # crawler content
                                res_sec_REJ = requests.post(url_each, headers=headers, data=data_REJ)
                                raw_content_REJ = json.loads(res_sec_REJ.text)
                                content_REJ = raw_content_REJ['content']['content']

                                # key:value
                                values = [title_REJ, kind_REJ, mark_REJ, content_REJ]
                                for n in range(len(keys)):
                                    d1[keys[n]] = values[n]

                                # save jsonfile
                                try:
                                    with open('%s/%s.json' % (paths[num], years[year]), 'a', encoding='utf-8') as f:
                                        json.dump(d1, f, ensure_ascii=False)
                                        f.write('\n')
                                except FileNotFoundError as e:
                                    no_REJ = no_REJ.replace('/', '-')
                                    with open('%s/%s.json' % (paths[num], years[year]), 'a', encoding='utf-8') as f:
                                        json.dump(d1, f, ensure_ascii=False)
                                        f.write('\n')
                                except OSError:
                                    pass
                                print("save json complete!===========================================================")

                                # print result
                                item_REJ += 1
                                if dkind[num] == "REJ":
                                    print(dkind[num], years[year], 'REJ: ', item_REJ, title_REJ)

                                # sleep
                                sleep(randint(5, 7))

                            except TypeError as e:
                                print('==========')
                                print(title_REJ, ': no content found')
                                print(e.args)
                                print('==========')
                                continue
                            except json.JSONDecodeError as e:
                                print('==========')
                                print(title_REJ, ':no page found')
                                print(e.args)
                                print('==========')
                                continue

                # date_29
                else: #months[month] in m29
                    lastday = 29
                    days_end = [10, 20, 25,29]
                    for dayend,daysta in zip(range(4),range(4)):
                        data_REJ = {'state': 'queryResultJSONObject',
                                         'searchConditions': '{"datePubStart":"%s/%s/%s","datePubEnd":"%s/%s/%s","dptKind":"%s","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'
                                         %(years[year], months[month], days_sta[daysta], years[year],months[month], days_end[dayend], dkind[num])
                                         }
                        res_REJ = requests.post(url_start, headers=headers, data=data_REJ)
                        list_REJ = json.loads(res_REJ.text)
                        #                     pp.pprint(list_REJ)
                        print('totalnum', years[year], "month:",months[month],"section:",daysta,list_REJ['receiveData']['totalNum'])
                        item_REJ = 0

                        for i in list_REJ['receiveData']['resultArr']:
                            d1 = {}
                            try:
                                kind_REJ = i['DPT_KIND']
                                mark_REJ = i['MARK_NAME']
                                no_REJ = i['PUB_NO']
                                title_REJ = no_REJ[-12:-1]
                                file_REJ = i['FILENAME']
                                data_REJ = {'state': 'getContentByFileName',
                                            'filename': file_REJ
                                            }

                                # crawler content
                                res_sec_REJ = requests.post(url_each, headers=headers, data=data_REJ)
                                raw_content_REJ = json.loads(res_sec_REJ.text)
                                content_REJ = raw_content_REJ['content']['content']

                                # key:value
                                values = [title_REJ, kind_REJ, mark_REJ, content_REJ]
                                for n in range(len(keys)):
                                    d1[keys[n]] = values[n]

                                # save jsonfile
                                try:
                                    with open('%s/%s.json' % (paths[num], years[year]), 'a', encoding='utf-8') as f:
                                        json.dump(d1, f, ensure_ascii=False)
                                        f.write('\n')
                                except FileNotFoundError as e:
                                    no_REJ = no_REJ.replace('/', '-')
                                    with open('%s/%s.json' % (paths[num], years[year]), 'a', encoding='utf-8') as f:
                                        json.dump(d1, f, ensure_ascii=False)
                                        f.write('\n')
                                except OSError:
                                    pass
                                print("save json complete!===========================================================")

                                # print result
                                item_REJ += 1
                                if dkind[num] == "REJ":
                                    print(dkind[num], years[year], 'REJ: ', item_REJ, title_REJ)

                                # sleep
                                sleep(randint(5, 7))

                            except TypeError as e:
                                print('==========')
                                print(title_REJ, ': no content found')
                                print(e.args)
                                print('==========')
                                continue
                            except json.JSONDecodeError as e:
                                print('==========')
                                print(title_REJ, ':no page found')
                                print(e.args)
                                print('==========')
                                continue

        except json.JSONDecodeError as e:
            print('==========')
            #         print(res_REJ.text)
            print(e.args)
            print('==========')
            continue
# sleep range (from 1 to 7)
# print(item)
# sleep(randint(1,7))

now is : kind REJ
now 0 mon 1
save json complete!===========================================================
REJ 102 REJ:  1 10280010950
save json complete!===========================================================
REJ 102 REJ:  2 10280010960
save json complete!===========================================================
REJ 102 REJ:  3 10280010970
save json complete!===========================================================
REJ 102 REJ:  4 10280010980
save json complete!===========================================================
REJ 102 REJ:  5 10280010990
save json complete!===========================================================
REJ 102 REJ:  6 10280011000
save json complete!===========================================================
REJ 102 REJ:  7 10280011010
save json complete!===========================================================
REJ 102 REJ:  8 10280011020
save json complete!===========================================================
REJ 102 REJ:  9 10280011030
save js

In [83]:
## download all by 1 ZONE 1
import requests
from bs4 import BeautifulSoup
import os
import json
import pprint as pp
import time
from random import randint
from time import sleep
import urllib.request 

ZONE4_path = r'./ZONE4/'
for i in range(102,110):
    if not os.path.exists(ZONE4_path):
        os.mkdir(ZONE4_path)
    if not os.path.exists("%s/%s"%(ZONE4_path,i)):
        os.mkdir("%s/%s"%(ZONE4_path,i))

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'} 

url_start = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=5f1da434-ad44-4955-97d3-1ac4433603f1(203.204.81.115)'
# url_start = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=f61d5b9e-0eb7-4160-86c9-ab76e064cb8c(111.249.61.96)'
url_each = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp'
dkind = ["REJ"] # "REJ" over 130K
paths = [ZONE4_path]
months = [1,2,3,4,5,6,7,8,9,10,11,12]
years = [102,103,104,105,106,107,108,109]
m31 = [1,3,5,7,8,10,12]
m30 = [4,6,9,11]
m29 = [2]

data_REJ = {}
# ZONE4_targets_fore = [399,349,384,359,307,386,384,273]
# ZONE4_targets_back = [496,410,374,381,342,429,358,192]

keys = ["title","kind","markname","content"]

for num in range(1):
#select type of judgement.
    print("now is : kind",dkind[num])

    # year period select
    for year in range(8):
        try:
            for month in range(12):
                
                # date_31
                if month in m31:
                    lastday = 31
                    data_REJ = {'state': 'queryResultJSONObject',
                'searchConditions': '{"datePubStart":"%s/%s/01","datePubEnd":"%s/%s/%s","dptKind":"%s","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years[year],months[month],years[year],months[month],lastday,dkind[num])
                       }
                # date_30
                if month in m30:
                    lastday = 30
                    data_REJ = {'state': 'queryResultJSONObject',
                'searchConditions': '{"datePubStart":"%s/%s/01","datePubEnd":"%s/%s/%s","dptKind":"%s","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years[year],months[month],years[year],months[month],lastday,dkind[num])
                       }
                #date_29
                if month in m29:
                    lastday = 29
                    data_REJ = {'state': 'queryResultJSONObject',
                'searchConditions': '{"datePubStart":"%s/%s/01","datePubEnd":"%s/%s/%s","dptKind":"%s","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years[year],months[month],years[year],months[month],lastday,dkind[num])
                       }
                
                
#                 print(data_REJ)
                res_REJ = requests.post(url_start, headers=headers, data=data_REJ)
                list_REJ = json.loads(res_REJ.text)
#                 pp.pprint(list_REJ)
                print('totalnum',years[year],months[month],list_REJ['receiveData']['totalNum'])                
                item_REJ = 0

#                 # crawler REJ year data
#                 for i in list_REJ['receiveData']['resultArr']:
#                     d1 = {}
#                     try:

#                         kind_REJ = i['DPT_KIND']
#                         mark_REJ = i['MARK_NAME']
#                         no_REJ = i['PUB_NO']
#                         title_REJ = no_REJ[-12:-1]
# #                         print(title_REJ)
#                         file_REJ = i['FILENAME']
#                         data_REJ  = {'state': 'getContentByFileName',
#                                     'filename': file_REJ
#                                      }

#                         # crawler content
#                         res_sec_REJ = requests.post(url_each, headers=headers, data=data_REJ)
#                         raw_content_REJ = json.loads(res_sec_REJ.text)
#     #                     pp.pprint(raw_content_REJ)
#                         content_REJ = raw_content_REJ['content']['content']

#                         #key:value
#                         values = [title_REJ,kind_REJ,mark_REJ,content_REJ]
#                         for n in range(len(keys)):
#                             d1[keys[n]]=values[n]

    #                     #save jsonfile
    #                     try:
    #                         with open('%s/%s.json' %(paths[num],years[year]), 'a', encoding='utf-8') as f:
    #                             json.dump(d1, f, ensure_ascii=False)
    #                             f.write('\n')
    #                     except FileNotFoundError as e:
    #                         no_REJ = no_REJ.replace('/', '-')
    #                         with open('%s/%s.json' %(paths[num],years[year]), 'a', encoding='utf-8') as f:
    #                             json.dump(d1, f, ensure_ascii=False)
    #                             f.write('\n')
    #                     except OSError:
    #                         pass
    #                     print("save json complete!===========================================================")

                        #print result
    #                         item_REJ +=1
    #                         if dkind[num] == "OPP":
    #                             print(dkind[num],years[year],'REJ: ',item_REJ,"/%s"%(ZONE1_targets_REJ[year]), title_REJ)

    #                     #sleep
    #                     sleep(randint(5,7))

#                     except TypeError as e:
#                         print('==========')
#                         print(title_REJ,': no content found')
#                         print(e.args)
#                         print('==========')
#                         continue
#                     except json.JSONDecodeError as e:
#                         print('==========')
#                         print(title_REJ, ':no page found')
#                         print(e.args)
#                         print('==========')
#                         continue

                #sleep range
#                 sleep(randint(10,15))

        except json.JSONDecodeError as e:
            print('==========')
    #         print(res_REJ.text)
            print(e.args)
            print('==========')
            continue
# sleep range (from 1 to 7)
# print(item)
# sleep(randint(1,7))

now is : kind REJ


KeyError: 'receiveData'